In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

In [2]:
split = 2

In [3]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/combosciplex/adata_with_predictions_{split}.h5ad")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [5]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [6]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [7]:
adata_pred_test = adata_preds[adata_preds.obs["split"]=="test"]
adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds[adata_preds.obs["split"]=="ood"]
adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

/tmp/ipykernel_1821627/2350350615.py:2: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
/tmp/ipykernel_1821627/2350350615.py:18: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]


In [8]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [9]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [10]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [11]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [12]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [13]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.619330814906529,
 'decoded_ood_sinkhorn_div_1': 117.77313123430524,
 'decoded_ood_sinkhorn_div_10': 84.00244631086078,
 'decoded_ood_sinkhorn_div_100': 37.18707166399275,
 'decoded_ood_e_distance': 73.46942435623205,
 'decoded_ood_mmd': 0.6508622169494629}

In [14]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7603089933594068,
 'decoded_test_e_distance': 47.37565842716649,
 'decoded_test_mmd_distance': 0.6311647295951843}

In [15]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.619330814906529,
 'decoded_ood_sinkhorn_div_1': 117.77313123430524,
 'decoded_ood_sinkhorn_div_10': 84.00244631086078,
 'decoded_ood_sinkhorn_div_100': 37.18707166399275,
 'decoded_ood_e_distance': 73.46942435623205,
 'decoded_ood_mmd': 0.6508622169494629}

In [16]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -2.770301648548671,
 'encoded_ood_sinkhorn_div_1': 41.11137580871582,
 'encoded_ood_sinkhorn_div_10': 36.95082262584141,
 'encoded_ood_sinkhorn_div_100': 35.56114346640451,
 'encoded_ood_e_distance': 70.79032980681617,
 'encoded_ood_mmd': 0.6645498786653791}

In [17]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.3485683628490993,
 'deg_ood_sinkhorn_div_1': 31.15391492843628,
 'deg_ood_sinkhorn_div_10': 22.274187360491073,
 'deg_ood_sinkhorn_div_100': 21.589465413774764,
 'deg_ood_e_distance': 43.03734289532476,
 'deg_ood_mmd': 0.649787894317082}

In [18]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.5834049805998802,
 'deg_test_sinkhorn_div_1': 22.848805050055187,
 'deg_test_sinkhorn_div_10': 14.360284407933554,
 'deg_test_sinkhorn_div_100': 13.303453465302786,
 'deg_test_e_distance': 26.449814610356242,
 'deg_test_mmd': 0.6063498382767042}

In [19]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7603089933594068,
 'decoded_test_e_distance': 47.37565842716649,
 'decoded_test_mmd_distance': 0.6311647295951843}

In [20]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.619330814906529,
 'decoded_ood_sinkhorn_div_1': 117.77313123430524,
 'decoded_ood_sinkhorn_div_10': 84.00244631086078,
 'decoded_ood_sinkhorn_div_100': 37.18707166399275,
 'decoded_ood_e_distance': 73.46942435623205,
 'decoded_ood_mmd': 0.6508622169494629}

In [21]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.3485683628490993,
 'deg_ood_sinkhorn_div_1': 31.15391492843628,
 'deg_ood_sinkhorn_div_10': 22.274187360491073,
 'deg_ood_sinkhorn_div_100': 21.589465413774764,
 'deg_ood_e_distance': 43.03734289532476,
 'deg_ood_mmd': 0.649787894317082}

In [22]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7603089933594068,
 'decoded_test_e_distance': 47.37565842716649,
 'decoded_test_mmd_distance': 0.6311647295951843}

In [23]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -0.20906823125754373,
 'encoded_test_sinkhorn_div_1': 28.688514073689777,
 'encoded_test_sinkhorn_div_10': 23.812433381875355,
 'encoded_test_sinkhorn_div_100': 22.20001721382141,
 'encoded_test_e_distance': 44.02279994203178,
 'encoded_test_mmd': 0.5969357726474603}

In [24]:
ood_metrics_encoded

{'Dacinostat+Dasatinib': {'r_squared': -5.824512004852295,
  'sinkhorn_div_1': 33.22028350830078,
  'sinkhorn_div_10': 29.48627471923828,
  'sinkhorn_div_100': 28.617206573486328,
  'e_distance': 57.05476535899118,
  'mmd': 0.69200563},
 'Dacinostat+PCI-34051': {'r_squared': -5.59621000289917,
  'sinkhorn_div_1': 33.9349365234375,
  'sinkhorn_div_10': 30.360679626464844,
  'sinkhorn_div_100': 29.525901794433594,
  'e_distance': 58.87158108834126,
  'mmd': 0.6939383},
 'Givinostat+Cediranib': {'r_squared': 0.9046927690505981,
  'sinkhorn_div_1': 7.0414018630981445,
  'sinkhorn_div_10': 3.0401220321655273,
  'sinkhorn_div_100': 1.526249885559082,
  'e_distance': 2.6797167688535004,
  'mmd': 0.4433876},
 'Givinostat+Curcumin': {'r_squared': 0.9898368120193481,
  'sinkhorn_div_1': 6.171200752258301,
  'sinkhorn_div_10': 2.0011839866638184,
  'sinkhorn_div_100': 0.3547687530517578,
  'e_distance': 0.2918890083036994,
  'mmd': 0.4252346},
 'Panobinostat+Alvespimycin': {'r_squared': -2.359155

In [25]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/chemcpa"

In [26]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
